In [3]:
from numba import jit

@jit(nopython=True)
def convert_pos(cub_idx, cell_idx):
    layer_cublet_idx = cub_idx % 100
    x_cub = layer_cublet_idx % 10
    y_cub = 9 - (layer_cublet_idx // 10)
    z_cub = cub_idx // 100

    layer_cell_idx = cell_idx % 100
    x_cell_temp = layer_cell_idx % 10
    y_cell_temp = 9 - (layer_cell_idx // 10)
    z_cell_temp = cell_idx // 100

    x_cell = x_cub * 10 + x_cell_temp
    y_cell = y_cub * 10 + y_cell_temp
    z_cell = z_cub * 10 + z_cell_temp

    return [x_cell, y_cell, z_cell]

In [8]:
# %%
import ROOT
import pandas as pd
from collections import defaultdict
import pickle
import time
from tqdm import tqdm

# Load the ROOT file and tree
file = ROOT.TFile("/mnt/c/Users/hnayak/Documents/Temp_pion/result_pion_1.root")
tree = file.Get("outputTree")
print(tree.Print())

# Function to process a single event and return a DataFrame

def process_event(event):
    # Define dictionaries to store total energy and sum of times for each cell
    cell_energy_dict = defaultdict(float)  # To store total energy per cell
    cell_time_dict = defaultdict(float)    # To store sum of times per cell
    cell_interactions_count = defaultdict(int)  # To count number of interactions per cell

    for j in range(event.Tinteractions_in_event):
        # Get cell coordinates (x, y, z)
        x, y, z = convert_pos(event.Tcublet_idx[j], event.Tcell_idx[j])
        
        # Get energy and time
        energy = event.Tedep[j]
        time = event.Tglob_t[j]  # Assuming this is the global time for the interaction
        
        # Accumulate total energy for the cell
        cell_energy_dict[(x, y, z)] += energy
        
        # Accumulate sum of time for the cell
        cell_time_dict[(x, y, z)] += time*energy
        
        # Count the number of interactions for the cell
        cell_interactions_count[(x, y, z)] += 1

    # Create a DataFrame for the current event
    df_event = pd.DataFrame([{
        "x": key[0],
        "y": key[1],
        "z": key[2],
        "total_energy": cell_energy_dict[key],
        "mean_time": cell_time_dict[key] / cell_interactions_count[key]  # Mean time (sum of time / interactions)
    } for key in cell_energy_dict.keys()])

    return df_event



None
******************************************************************************
*Tree    :outputTree: Output Tree Description                                *
*Entries :     1000 : Total =      4070149617 bytes  File  Size = 2307893723 *
*        :          : Tree compression factor =   1.76                       *
******************************************************************************
*Br    0 :Tevent_id : Tevent_id/I                                            *
*Entries :     1000 : Total  Size=      12175 bytes  File Size  =      10078 *
*Baskets :       77 : Basket Size=       6654 bytes  Compression=   1.02     *
*............................................................................*
*Br    1 :Tinteractions_in_event : Tinteractions_in_event/I                  *
*Entries :     1000 : Total  Size=      13228 bytes  File Size  =      11315 *
*Baskets :       77 : Basket Size=       6654 bytes  Compression=   1.00     *
*..............................................

In [9]:
# List to store DataFrames for each event
df_events = []
total_entries = tree.GetEntries()
# Loop through the events in the tree

for i, event in enumerate(tqdm(tree, total=total_entries, desc="Processing Events", unit="event", ascii=True)):
    # if i % 10 == 0:
    #     print(f"Processing event: {i} / {total_entries}")
    if i > 10:
        break
    df_event = process_event(event)
    df_events.append(df_event)
    
    # Optional: Simulate some processing time (can remove in real code)
    time.sleep(0.01)

df_events

Processing Events:   1%|1         | 11/1000 [00:06<09:16,  1.78event/s]


[       x   y  z  total_energy  mean_time
 0     50  49  0      6.805836   9.217089
 1     50  49  1      4.344247   0.175955
 2     50  50  1      7.033426   1.376197
 3     50  50  2      5.375503   7.495173
 4     50  50  3      3.061828   2.149007
 ...   ..  .. ..           ...        ...
 9087  50  48  4      0.037870   0.027297
 9088  51  48  4      0.540258   0.195789
 9089  50  49  3      0.187697   0.066057
 9090  49  49  3      1.821722   0.111679
 9091  51  49  1      0.508151   0.050107
 
 [9092 rows x 5 columns],
         x   y  z  total_energy  mean_time
 0      50  49  0      5.487479   7.431649
 1      50  49  1      0.013556   0.018360
 2      50  50  1      0.020749   0.028103
 3      49  50  1      6.236333   8.570631
 4      49  50  2      5.710468   7.962221
 ...    ..  .. ..           ...        ...
 11468  50  49  5      0.008905   0.012939
 11469  49  49  5      0.001767   0.002567
 11470  49  50  5      0.478543   0.024029
 11471  52  53  7      0.130659   0.09

In [ ]:
with open("/mnt/Proton1.pkl","wb") as f:
    pickle.dump(df_events, f)
    f.close()
print("Proton1.pkl created")

In [2]:
def convert_pos(cub_idx, cell_idx, size=(100, 100, 200)):
    """
    Converts cublet and cell indices into 3D grid positions with scaling.
    """
    layer_cublet_idx = cub_idx % 100

    x_cub = layer_cublet_idx % 10
    y_cub = 9 - (layer_cublet_idx // 10)
    z_cub = cub_idx // 100

    layer_cell_idx = cell_idx % 100

    x_cell_temp = layer_cell_idx % 10
    y_cell_temp = 9 - (layer_cell_idx // 10)
    z_cell_temp = cell_idx // 100

    # Convert the 1D index into 3D coordinates (x, y, z)
    x_cell = x_cub * 10 + x_cell_temp
    y_cell = y_cub * 10 + y_cell_temp
    z_cell = z_cub * 10 + z_cell_temp  

    # Define the correct grid dimensions
    grid_x = 100
    grid_y = 100
    grid_z = 200  # Updated to 200 layers

    # Scale the coordinates to the new grid size
    new_x = x_cell // (grid_x // size[0])
    new_y = y_cell // (grid_y // size[1])
    new_z = z_cell // (grid_z // size[2])  # Correct scaling for 200 layers

    return [new_x, new_y, new_z]


In [12]:
convert_pos(21, 20000)

[10, 79, 200]